# 03. Data Preprocessing | المعالجة المسبقة للبيانات

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 1: Data Loading and Exploration** - Know your data structure
- ✅ **Example 2: Data Cleaning** - Have clean data to preprocess
- ✅ **Basic ML concepts**: Features, targets, training vs testing

**If you haven't completed these**, you might struggle with:
- Understanding why preprocessing is needed
- Knowing which preprocessing method to use
- Understanding the difference between scaling methods

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the THIRD example** - it prepares clean data for machine learning!

**Why this example THIRD?**
- **Before** you can build ML models, you need preprocessed data
- **Before** you can train models, you need scaled features
- **Before** you can make predictions, you need encoded categories

**Builds on**: 
- 📓 Example 1: Data Loading (we know our data)
- 📓 Example 2: Data Cleaning (we have clean data)

**Leads to**: 
- 📓 Example 4: Linear Regression (needs preprocessed data)
- 📓 Example 5: Polynomial Regression (needs scaled features)
- 📓 All ML models (all need preprocessing!)

**Why this order?**
1. Preprocessing transforms **clean data into ML-ready format**
2. Preprocessing teaches you **scaling vs encoding** (different problems, different solutions)
3. Preprocessing shows you **train-test split** (essential for evaluation)

---

## The Story: Preparing Ingredients for Cooking | القصة: تحضير المكونات للطبخ

Imagine you're cooking. **Before** you can cook, you need to prepare ingredients - cut vegetables to the same size, marinate meat, measure spices. **After** preparing everything uniformly, you can cook a perfect meal!

Same with machine learning: **Before** building models, we preprocess data - scale features to same range, encode categories to numbers, split into train/test. **After** preprocessing, we can build accurate models!

---

## Why Data Preprocessing Matters | لماذا تهم المعالجة المسبقة للبيانات؟

Preprocessing is essential for ML success:
- **Feature Scaling**: Algorithms work better when features are on similar scales
- **Encoding**: ML algorithms need numbers, not text categories
- **Train-Test Split**: We need separate data to evaluate model performance
- **Without Preprocessing**: Models fail or perform poorly

## Learning Objectives | أهداف التعلم
1. Scale features using Standardization and Normalization
2. Encode categorical variables (Label vs One-Hot)
3. Split data into training and testing sets
4. Understand when to use each preprocessing method
5. Build a complete preprocessing pipeline

In [1]:
# Step 1: Import necessary libraries
# These libraries help us preprocess data for machine learning

import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
from sklearn.preprocessing import (
    StandardScaler,    # For standardization (mean=0, std=1)
    MinMaxScaler,      # For normalization (range 0-1)
    LabelEncoder,      # For ordinal encoding
    OneHotEncoder      # For nominal encoding
)
from sklearn.model_selection import train_test_split  # For splitting data

print("✅ Libraries imported successfully!")
print("\n📚 What each preprocessing tool does:")
print("   - StandardScaler: Scale features to mean=0, std=1 (z-score)")
print("   - MinMaxScaler: Scale features to range [0, 1]")
print("   - LabelEncoder: Convert categories to numbers (ordinal)")
print("   - OneHotEncoder: Convert categories to binary columns (nominal)")
print("   - train_test_split: Split data into train/test sets")

## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We have clean data, but it's not ready for machine learning - features have different scales, categories are text, and we haven't split the data.

**AFTER**: We'll preprocess the data - scale features, encode categories, split into train/test - making it ready for ML models!

**Why this matters**: Most ML algorithms require preprocessed data. Without it, models fail or give poor results!

---

## Step 1: Load Real-World Data | الخطوة 1: تحميل بيانات العالم الحقيقي

**BEFORE**: We need to learn preprocessing, but we need real data with different scales and categories.

**AFTER**: We'll load the California Housing dataset - real data with features on different scales and we'll add categorical features to practice preprocessing techniques!

**Why use California Housing?** This is REAL data from the 1990 census with features that need scaling (MedInc ranges differently than HouseAge, etc.). Real datasets have these characteristics - we need to learn how to handle them.

In [ ]:
# Load real-world California Housing dataset for preprocessing
# This dataset has features on different scales and is perfect for learning preprocessing techniques!
from sklearn.datasets import fetch_california_housing

print("\n📥 Loading California Housing dataset...")
housing_data = fetch_california_housing()

# Create DataFrame
df = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
df['target'] = housing_data.target

# For preprocessing demonstration, we'll add a categorical column (high/medium/low income areas)
# This helps demonstrate encoding techniques
df['income_level'] = pd.cut(df['MedInc'], bins=[0, 2, 4, 10], labels=['Low', 'Medium', 'High'])

print(f"\n✅ Real-world California Housing data loaded!")
print(f"   📊 This is REAL data from the 1990 California census")
print(f"   📈 Contains {len(df)} housing districts with {len(df.columns)} features")
print(f"   🔍 Features have different scales (need standardization/normalization)")
print(f"   📋 Added 'income_level' category for encoding demonstration")

# Data is already loaded above from California Housing
# Let's inspect the data to see why preprocessing is needed

print("\n📊 Original Data (California Housing):")
print(df.head())
print(f"\n📐 Data Shape: {df.shape}")
print("\n🔍 Notice:")
print("   - Numeric features have VERY different scales:")
print(f"     - MedInc ranges: {df['MedInc'].min():.2f} to {df['MedInc'].max():.2f}")
print(f"     - HouseAge ranges: {df['HouseAge'].min():.1f} to {df['HouseAge'].max():.1f}")
print(f"     - AveRooms ranges: {df['AveRooms'].min():.2f} to {df['AveRooms'].max():.2f}")
print("   - Categorical feature (income_level) is text: Low, Medium, High")
print("   - ML algorithms need same-scale numbers! This is why we preprocess!")


In [3]:
print("\n" + "=" * 60)
print("2. Feature Scaling - Standardization")
print("قياس الميزات - التوحيد المعياري")
print("=" * 60)


## Step 2: Feature Scaling - Standardization | الخطوة 2: قياس الميزات - التوحيد المعياري

**BEFORE**: Features have very different scales (age: 20-60, salary: 30k-100k). Algorithms will be biased toward larger numbers!

**AFTER**: We'll standardize features so they all have mean=0 and std=1, putting them on the same scale!

**Why Standardization?**
- **Problem**: Features have different scales (age: 20-60, salary: 30k-100k)
- **Issue**: ML algorithms treat larger numbers as more important → biased toward salary!
- **Solution**: Standardization puts all features on same scale (mean=0, std=1)
- **Result**: All features contribute equally to the model
- **Formula**: `(x - mean) / std`

**Why mean=0 and std=1? (Common Student Questions)**

**Q: Why mean=0, not 1 or 100?**
- **Answer**: We subtract the mean → `(x - mean)` centers data at 0
  - If mean=40: value=50 becomes (50-40)=10, value=30 becomes (30-40)=-10
  - The average of all (x - mean) values is always 0 (by definition!)
  - Mean=0 means data is centered (balanced around zero)
  - Why not 1? Because subtracting mean naturally gives 0, not 1

**Q: Why std=1, not 0 or 2?**
- **Answer**: We divide by std → `(x - mean) / std` normalizes spread to 1
  - If std=10: value=50 becomes (50-40)/10=1.0, value=30 becomes (30-40)/10=-1.0
  - Dividing by std makes the new std exactly 1 (by definition!)
  - Std=1 means "one unit of variation" (standard unit)
  - Why not 0? Std=0 means no variation (all values same) - not useful!
  - Why not 2? We want standard unit (1), not arbitrary number

**Q: Why these specific numbers (0 and 1)?**
- **Answer**: They're the "standard" values that make comparison easy
  - Mean=0: Easy to see if value is above/below average (positive/negative)
  - Std=1: Easy to see how many standard deviations away (1.5 = 1.5 std away)
  - Together: All features use same "standard scale" for fair comparison

- **Use when**: Data is normally distributed, outliers are important
- **Good for**: Linear models, neural networks, distance-based algorithms

**Note**: Remove problematic outliers first (Example 2), then standardize. Standardization preserves outliers, so clean data first!

In [4]:
# Create StandardScaler object
# Standardization formula: (x - mean) / std
# This transforms data so mean=0 and std=1

# StandardScaler()
# - Creates scaler object for standardization
# - Standardization: (x - mean) / std
#   - Transforms data so mean = 0, standard deviation = 1
#   - Centers data around 0, scales to unit variance
# - Two-step process:
#   1. .fit(): Learn mean and std from data
#   2. .transform(): Apply transformation
# .fit_transform(data)
# - Two operations in one: .fit() then .transform()
#   1. .fit(): Learns parameters from data (mean/std, categories, etc.)
#   2. .transform(): Applies transformation using learned parameters
# - Use on training data
# - For test data, use only .transform() (don't refit!)

# - Or use .fit_transform(): Do both in one step
# - Important: Fit on training data, transform both train and test
scaler_standard = StandardScaler()
print("   ✅ StandardScaler created")
print("   Formula: (x - mean) / std")

In [5]:
# Select numeric columns for scaling
# Why only numeric? Categorical columns need encoding, not scaling!
numeric_cols = ['age', 'salary', 'experience']
df_scaled_standard = df.copy()
print(f"   Scaling columns: {numeric_cols}")

In [6]:
# Fit and transform the data
# .fit_transform() learns the mean/std from data, then transforms it
# Why fit_transform? We learn parameters from training data, then apply to all data

# scaler_standard.fit_transform(df[numeric_cols])
# - .fit_transform(): Two operations in one
#   1. .fit(): Learns mean and std from data
#      - Calculates mean and std for each column
#      - Stores these values in scaler object
#   2. .transform(): Applies standardization using learned parameters
#      - Formula: (x - learned_mean) / learned_std
#      - Transforms each value in column
# - df[numeric_cols]: Selects only numeric columns to scale
#   - List of column names: ['age', 'salary', 'experience']
#   - Returns DataFrame with only those columns
# - Result: All numeric columns now have mean≈0 and std≈1
# - Note: In real ML, fit on training data, transform on both train and test
df_scaled_standard[numeric_cols] = scaler_standard.fit_transform(df[numeric_cols])

print("\n✅ After Standardization (mean=0, std=1):")
print("بعد التوحيد المعياري (متوسط=0، انحراف معياري=1):")
print(f"   Age - Mean: {df_scaled_standard['age'].mean():.4f}, Std: {df_scaled_standard['age'].std():.4f}")
print(f"   Salary - Mean: {df_scaled_standard['salary'].mean():.4f}, Std: {df_scaled_standard['salary'].std():.4f}")
print(f"   Experience - Mean: {df_scaled_standard['experience'].mean():.4f}, Std: {df_scaled_standard['experience'].std():.4f}")
print("\n   ✅ All features now have mean≈0 and std≈1!")
print("\n   📝 Understanding the values:")
print("      - Positive values (e.g., 1.67): Above average (1.67 std above mean)")
print("      - Negative values (e.g., -1.36): Below average (1.36 std below mean)")
print("      - Values near 0: Close to average")
print("      - Values > 2 or < -2: Far from average (potential outliers)")
print("\n📄 Scaled data (first 5 rows):")
print(df_scaled_standard[numeric_cols].head())

## Step 3: Feature Scaling - Normalization (Min-Max) | الخطوة 3: قياس الميزات - التطبيع

**BEFORE**: Features have different ranges. We want them all in the same range [0, 1].

**AFTER**: We'll normalize features so they all range from 0 to 1!

**Why Normalization (Min-Max)?**
- **Problem**: Same as standardization - different scales
- **Solution**: Normalization puts all features in range [0, 1]
- **Formula**: `(x - min) / (max - min)`
- **Result**: Range [0, 1] (minimum becomes 0, maximum becomes 1)

**Common Student Questions:**
- **Q: Why range [0, 1], not [0, 100]?**
  - Answer: [0, 1] is standard range (0% to 100% of the spread)
  - Easy to interpret: 0 = minimum, 1 = maximum, 0.5 = middle
- **Q: Why not use standardization?**
  - Answer: Normalization gives bounded [0,1] range (good for neural networks)
  - Standardization can give values outside [0,1] (e.g., -2.5, 3.1)
- **Q: Which is better?**
  - Answer: Depends on your algorithm!
  - Standardization: Better for normal data, preserves outliers
  - Normalization: Better for neural networks, bounded range needed

- **Use when**: Data is not normally distributed, you want bounded values
- **Good for**: Neural networks, algorithms that need [0,1] range

In [7]:
# Normalization: (x - min) / (max - min) -> range [0, 1]
# التطبيع: (x - الأدنى) / (الأعلى - الأدنى) -> النطاق [0, 1]

# MinMaxScaler()
# - Creates scaler object for min-max normalization
# - Normalization: (x - min) / (max - min)
#   - Transforms data to range [0, 1]
#   - Minimum value becomes 0, maximum becomes 1
# - Two-step process:
#   1. .fit(): Learn min and max from data
#   2. .transform(): Apply transformation
# - Or use .fit_transform(): Do both in one step
# - Use when: Data not normally distributed, need bounded [0,1] range
scaler_minmax = MinMaxScaler()
df_scaled_minmax = df.copy()
df_scaled_minmax[numeric_cols] = scaler_minmax.fit_transform(df[numeric_cols])
print("\nAfter Min-Max Normalization (range 0-1):")
print("بعد التطبيع بالأدنى-الأعلى (النطاق 0-1):")
print(f"Age - Min: {df_scaled_minmax['age'].min():.4f}, Max: {df_scaled_minmax['age'].max():.4f}")
print(f"Salary - Min: {df_scaled_minmax['salary'].min():.4f}, Max: {df_scaled_minmax['salary'].max():.4f}")
print(df_scaled_minmax[numeric_cols].head())

## Step 4: Label Encoding | الخطوة 4: الترميز بالعلامات

**BEFORE**: We have categorical text data (e.g., 'Bachelor', 'Master', 'PhD'), but ML algorithms need numbers!

**AFTER**: We'll encode categories to numbers using Label Encoding - perfect for ordinal categories (categories with order)!

**Common Student Questions:**
- **Q: What is ordinal vs nominal?**
  - Answer: Ordinal = categories have order (e.g., Low < Medium < High, Bachelor < Master < PhD)
  - Nominal = categories have no order (e.g., city names, colors, departments)
  - Label encoding preserves order → use for ordinal
  - One-hot encoding treats all equally → use for nominal
- **Q: Why use Label Encoding instead of One-Hot?**
  - Answer: Label encoding preserves the order relationship
  - Example: Bachelor=0, Master=1, PhD=2 → algorithm knows PhD > Master > Bachelor
  - One-hot would create 3 separate columns → loses the order information
  - Use label encoding when order matters (education levels, ratings, sizes)
- **Q: Can I use Label Encoding for nominal categories?**
  - Answer: Technically yes, but NOT recommended!
  - Problem: Creates false order (e.g., 'Red'=0, 'Blue'=1, 'Green'=2 → algorithm thinks Green > Blue > Red)
  - Solution: Use One-Hot Encoding for nominal categories (no false order)
- **Q: How does LabelEncoder assign numbers?**
  - Answer: Assigns numbers in alphabetical order (or order found in data)
  - Example: ['Bachelor', 'Master', 'PhD'] → Bachelor=0, Master=1, PhD=2
  - Important: Order might not match your desired order → check `label_encoder.classes_`
- **Q: What if I want a specific order?**
  - Answer: You can manually map categories to numbers using a dictionary
  - Example: `df['education'] = df['education'].map({'Bachelor': 0, 'Master': 1, 'PhD': 2})`
  - Or use pandas `Categorical` with `ordered=True` to control order

**Use when**: Categories have inherent order (ordinal data)
**Good for**: Education levels, ratings, sizes, rankings


In [8]:

# LabelEncoder()
# - Creates encoder for ordinal categorical data
# - Ordinal: Categories have order (e.g., Bachelor < Master < PhD)
# - Converts text categories to integers (0, 1, 2, ...)
# - Two-step process:
#   1. .fit(): Learn unique categories
#   2. .transform(): Convert categories to numbers
# - Or use .fit_transform(): Do both in one step
# - Use when: Categories have meaningful order
label_encoder = LabelEncoder()

In [9]:
# LabelEncoder()
# - Creates encoder for ordinal categorical data
# - Ordinal: Categories have order (e.g., Bachelor < Master < PhD)
# - Converts text categories to integers (0, 1, 2, ...)
# - Two-step process:
#   1. .fit(): Learn unique categories
#   2. .transform(): Convert categories to numbers
# - Or use .fit_transform(): Do both in one step
# - Use when: Categories have meaningful order
label_encoder = LabelEncoder()

# Encode education (if we assume ordinal: Bachelor < Master < PhD)
df_encoded = df.copy()

# label_encoder.fit_transform(df['education'])
# - .fit_transform(): Two operations in one
#   1. .fit(): Learns unique categories from data
#      - Finds all unique values (e.g., 'Bachelor', 'Master', 'PhD')
#      - Assigns numbers in alphabetical order (or order found)
#   2. .transform(): Converts categories to integers
#      - 'Bachelor' → 0, 'Master' → 1, 'PhD' → 2
# - df['education']: Passes education column as Series
# - Returns numpy array with encoded integers
# - label_encoder.classes_: Shows mapping (original categories)
df_encoded['education_encoded'] = label_encoder.fit_transform(df['education'])

# label_encoder.classes_
# - Returns array of unique categories in order they were encoded
# - Used to see mapping: classes_[0] = first category, etc.
print("\nLabel Encoding for Education:")
print("الترميز بالعلامات للتعليم:")
print("Mapping / التعيين:")
for i, label in enumerate(label_encoder.classes_):
    print(f"  {label}: {i}")
print("\nEncoded values:")
print(df_encoded[['education', 'education_encoded']].head(10))

## Step 5: One-Hot Encoding | الخطوة 5: الترميز الواحد-الساخن

**BEFORE**: We have nominal categories (no order, like city names), but ML algorithms need numbers!

**AFTER**: We'll encode categories using One-Hot Encoding - creates separate binary columns for each category!

**Common Student Questions:**
- **Q: What is One-Hot Encoding?**
  - Answer: Each category becomes a separate binary column (0 or 1)
  - Example: City ['Riyadh', 'Jeddah', 'Dammam'] → 3 columns: city_Riyadh, city_Jeddah, city_Dammam
  - Each row has 1 in one column (the city), 0 in all others
  - No false order → perfect for nominal categories
- **Q: Why use One-Hot instead of Label Encoding?**
  - Answer: One-Hot treats all categories equally (no false order)
  - Label encoding creates false order for nominal data (e.g., 'Red'=0, 'Blue'=1 → algorithm thinks Blue > Red)
  - One-Hot: Each category is independent → no order assumptions
  - Use One-Hot for: City names, colors, departments, any categories without order
- **Q: Why does One-Hot create so many columns?**
  - Answer: Each category needs its own column to represent it independently
  - Example: 10 cities → 10 columns (or 9 if using drop='first')
  - Trade-off: More columns = more memory, but no false order
  - Solution: For many categories (>50), consider Target Encoding or Frequency Encoding
- **Q: What is drop='first' in sklearn OneHotEncoder?**
  - Answer: Drops the first category column to avoid multicollinearity
  - Problem: If you have N categories, you only need N-1 columns (one is redundant)
  - Example: 4 cities → 3 columns (can infer 4th from the other 3)
  - Why: Prevents perfect correlation between columns (helps with linear models)
- **Q: When should I use pandas get_dummies vs sklearn OneHotEncoder?**
  - Answer: pandas get_dummies = easier, good for quick work
  - sklearn OneHotEncoder = more control (drop='first', sparse matrices, better for pipelines)
  - Use pandas: Quick prototyping, simple cases
  - Use sklearn: Production code, need drop='first', building pipelines

**Use when**: Categories have no inherent order (nominal data)
**Good for**: City names, colors, departments, any categories without order

In [16]:
# Using pandas get_dummies (easier)
# استخدام pandas get_dummies (أسهل)

# pd.get_dummies(df, columns=['city', 'education'], prefix=['city', 'edu'])
# - pd.get_dummies(): Creates one-hot encoded columns for categorical data
# - df: DataFrame to encode
# - columns=['city', 'education']: Columns to encode (list of column names)
# - prefix=['city', 'edu']: Prefix for new column names
#   - 'city' column with values ['Riyadh', 'Jeddah'] → 'city_Riyadh', 'city_Jeddah'
#   - 'education' column → 'edu_Bachelor', 'edu_Master', 'edu_PhD'
# - One-hot encoding: Each category becomes a binary column (0 or 1)
#   - Example: If city='Riyadh', then city_Riyadh=1, others=0
# - Returns DataFrame with original columns + new binary columns
# - Use for: Nominal categories (no order, like city names)
df_onehot = pd.get_dummies(df, columns=['city', 'education'], prefix=['city', 'edu'])

# df_onehot.columns.tolist()
# - df_onehot.columns: Returns Index with column names
# - .tolist(): Converts to Python list
# - Shows all column names after encoding
print("\nOne-Hot Encoded columns:")
print("الأعمدة المشفرة بالواحد-الساخن:")
print(df_onehot.columns.tolist())
print("\nSample of One-Hot Encoded data:")
print(df_onehot[['age', 'salary'] + [col for col in df_onehot.columns if 'city_' in col or 'edu_' in col]].head())

In [11]:
# Method 2: Using sklearn OneHotEncoder (for more control)
# Why sklearn? More control - can drop first column to avoid multicollinearity
# drop='first' removes one column (redundant - can be inferred from others)

print("\n--- Using sklearn OneHotEncoder ---")

# OneHotEncoder(sparse_output=False, drop='first')
# - OneHotEncoder(): Creates one-hot encoder object
# - sparse_output=False: Returns dense array (True returns sparse matrix)
# - drop='first': Drops first category column to avoid multicollinearity
#   - If 4 cities, creates 3 columns (not 4)
#   - One column is redundant (can be inferred from others)
#   - Prevents perfect correlation between columns
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')

# onehot_encoder.fit_transform(df[['city']])
# - .fit_transform(): Learns categories and encodes
#   1. .fit(): Learns unique categories
#   2. .transform(): Creates binary columns
# - df[['city']]: Double brackets [[]] to pass as DataFrame (not Series)
#   - OneHotEncoder expects 2D input (DataFrame or 2D array)
# - Returns numpy array with binary columns
# - Example: 4 cities → 3 columns (if drop='first')
city_encoded = onehot_encoder.fit_transform(df[['city']])

print("   City encoding classes:")
print(f"   {onehot_encoder.categories_}")
print(f"\n   Encoded shape: {city_encoded.shape}")
print(f"   (4 cities → 3 columns, because drop='first' removes one)")
print("\n   First 5 encoded values:")
print(city_encoded[:5])
print("\n   ✅ Each row has 1 in one column (the city), 0 in others")

## Step 6: Train-Test Split | الخطوة 6: تقسيم البيانات إلى تدريب واختبار

**BEFORE**: We have all our data together. We can't evaluate model performance on data it has seen!

**AFTER**: We'll split data into training (80%) and testing (20%) sets!

**Why Train-Test Split?**
- **Training set**: Used to train/learn the model
- **Test set**: Used to evaluate model performance (unseen data)
- **Why separate?** Testing on training data gives false high scores (overfitting)
- **Standard split**: 80% train, 20% test (can vary based on dataset size)

**Common Student Questions:**
- **Q: Why split data into train and test?**
  - Answer: Need unseen data to evaluate model performance honestly
  - Problem: Testing on training data = model memorized data → gives false high scores
  - Solution: Train on 80%, test on 20% → see how model performs on new data
  - Real-world: Model must work on data it hasn't seen before!
- **Q: Why 80/20 split?**
  - Answer: Good balance between training (need enough to learn) and testing (need enough to evaluate)
  - 80% training: Enough data for model to learn patterns
  - 20% testing: Enough data for reliable performance evaluation
  - Can adjust: Small datasets (<1000) → 70/30, Large datasets (>10k) → 90/10
- **Q: What is random_state?**
  - Answer: Seed for random number generator → ensures same split every time
  - Problem: Without seed, different split each run → can't reproduce results
  - Solution: Set random_state=42 → same split every time (reproducibility)
  - Important: Use same random_state for all experiments to compare fairly
- **Q: What is stratify=y?**
  - Answer: Maintains class distribution in train/test sets
  - Problem: Random split might create imbalanced train/test (e.g., 60% class 0 in train, 40% in test)
  - Solution: stratify=y → train and test have same class distribution
  - Example: If 60% class 0, 40% class 1 → both train and test have 60/40 split
  - Use when: Classification problems with imbalanced classes
- **Q: Should I split before or after preprocessing?**
  - Answer: Split FIRST, then preprocess train and test separately!
  - Why: Fit scaler/encoder on training data only, then transform both train and test
  - Problem: Preprocessing before split = data leakage (test data influences training)
  - Correct order: Split → Fit on train → Transform train and test
  - Note: In this notebook, we show preprocessing on full data for simplicity, but in real projects, split first!

In [12]:
# Prepare features and target
X = df[['age', 'salary', 'experience']]
y = df['target']

In [13]:
# Split into train and test sets (80% train, 20% test)
# التقسيم إلى مجموعات تدريب واختبار (80% تدريب، 20% اختبار)

# train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# - train_test_split(): Splits data into training and testing sets
# - X: Features (input variables) - DataFrame or array
# - y: Target (output variable) - Series or array
# - test_size=0.2: 20% for testing, 80% for training (can use 0.25 for 25%)
# - random_state=42: Seed for random number generator
#   - Ensures same split every time (reproducibility)
#   - Different seeds = different splits
# - stratify=y: Maintains class distribution in train/test
#   - If y has 60% class 0, 40% class 1, train/test will have same ratio
#   - Important for imbalanced datasets
# - Returns 4 arrays: X_train, X_test, y_train, y_test
#   - X_train: Training features (80% of X)
#   - X_test: Testing features (20% of X)
#   - y_train: Training targets (80% of y)
#   - y_test: Testing targets (20% of y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class distribution
)
print(f"\nOriginal data shape: {X.shape}")
print(f"شكل البيانات الأصلية: {X.shape}")
print(f"\nTraining set:")
print(f"مجموعة التدريب:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"\nTest set:")
print(f"مجموعة الاختبار:")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"\nTrain percentage: {len(X_train) / len(X) * 100:.1f}%")
print(f"Test percentage: {len(X_test) / len(X) * 100:.1f}%")

## Step 7: Complete Preprocessing Pipeline | الخطوة 7: خط أنابيب المعالجة الكاملة

**BEFORE**: We've learned individual preprocessing steps, but how do we combine them all together?

**AFTER**: We'll create a complete preprocessing pipeline that combines all steps - encoding, scaling, and splitting - into one reusable function!

**Why this matters**: In real projects, you need to apply the same preprocessing to training and test data consistently!

In [14]:
# مثال خط أنابيب المعالجة الكاملة
print("\n" + "=" * 60)
print("7. Complete Preprocessing Pipeline")
print("خط أنابيب المعالجة الكاملة")
print("=" * 60)

def preprocess_data(df, numeric_cols, categorical_cols, target_col, test_size=0.2):
    """
    Complete preprocessing pipeline
    خط أنابيب المعالجة الكاملة
    """
    # Separate features and target
    X = df[numeric_cols + categorical_cols].copy()
    y = df[target_col].copy()
    
    # One-hot encode categorical variables
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    
    # Scale numeric features
    scaler = StandardScaler()
    X_encoded[numeric_cols] = scaler.fit_transform(X_encoded[numeric_cols])
    
    # Split data
# train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# - Splits data into training and testing sets
# - X: Features (input variables), y: Target (output variable)
# - test_size=0.2: 20% for testing, 80% for training
# - random_state=42: Seed for reproducibility (same split every time)
# - stratify=y: Maintains class distribution in train/test (for classification)
# - Returns: X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=test_size, random_state=42, stratify=y
    )
    
    return X_train, X_test, y_train, y_test, scaler

## Step 8: Decision Framework - When to Use Each Preprocessing Method | الخطوة 8: إطار القرار - متى تستخدم كل طريقة معالجة

**BEFORE**: You've learned different preprocessing methods, but when should you use each one?

**AFTER**: You'll have a clear decision framework to choose the right preprocessing method for any situation!

**Why this matters**: Using the wrong preprocessing method can:
- **Break your models** → Wrong scaling causes algorithm failures
- **Reduce performance** → Inappropriate encoding loses information
- **Waste time** → Trying all methods without guidance

---

### 🎯 Decision Framework for Feature Scaling | إطار القرار لقياس الميزات

**Key Question**: Should I use **STANDARDIZATION** or **NORMALIZATION**?

#### Decision Tree:

```
Do you have numeric features?
├─ NO → No scaling needed (categorical features need encoding, not scaling)
│
└─ YES → Are features on different scales?
    ├─ NO → No scaling needed (already on similar scales)
    │
    └─ YES → Is data normally distributed?
        ├─ YES → Use STANDARDIZATION (StandardScaler)
        │   └─ Why? Preserves distribution, handles outliers well
        │
        └─ NO → Do you need bounded values [0, 1]?
            ├─ YES → Use NORMALIZATION (MinMaxScaler)
            │   └─ Why? Neural networks, algorithms requiring [0,1] range
            │
            └─ NO → Use STANDARDIZATION (StandardScaler)
                └─ Why? More robust to outliers, preserves relationships
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Standardization** | Normal distribution, outliers present, linear models | • Preserves distribution<br>• Robust to outliers<br>• Mean=0, Std=1 | • Can produce values outside [0,1]<br>• Assumes normal distribution | Age (20-60), Salary (30k-100k) |
| **Normalization** | Non-normal distribution, neural networks, need [0,1] | • Bounded [0,1]<br>• Works with any distribution<br>• Good for neural networks | • Sensitive to outliers<br>• May compress data too much | Image pixels (0-255), Ratings (1-5) |
| **No Scaling** | Features already on similar scales, tree-based models | • No transformation needed<br>• Preserves original values | • Only works if scales are similar | All features 0-1, or all 100-200 |

---

### 🎯 Decision Framework for Categorical Encoding | إطار القرار لترميز البيانات الفئوية

**Key Question**: Should I use **LABEL ENCODING** or **ONE-HOT ENCODING**?

#### Decision Tree:

```
Do you have categorical features?
├─ NO → No encoding needed (all features are numeric)
│
└─ YES → Is there an inherent order (ordinal)?
    ├─ YES → Use LABEL ENCODING
    │   └─ Why? Preserves order (e.g., Low < Medium < High)
    │
    └─ NO → How many unique categories?
        ├─ < 10 → Use ONE-HOT ENCODING
        │   └─ Why? Creates separate columns, no false order
        │
        └─ ≥ 10 → Consider alternatives:
            ├─ Use ONE-HOT if important features
            │   └─ Why? Each category matters
            │
            └─ Use TARGET ENCODING or frequency encoding
                └─ Why? Avoids creating too many columns
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Label Encoding** | Ordinal categories (inherent order) | • Preserves order<br>• Single column<br>• Simple | • Creates false order for nominal<br>• Algorithms may misinterpret | Education: Bachelor=0, Master=1, PhD=2 |
| **One-Hot Encoding** | Nominal categories (no order), < 10 categories | • No false order<br>• Each category separate<br>• Works with all algorithms | • Creates many columns<br>• Can cause curse of dimensionality | Department: IT, HR, Finance → 3 columns |
| **Target Encoding** | Many categories (>10), nominal | • Keeps single column<br>• Captures target relationship | • Can overfit<br>• More complex | City with 50+ values |

---

### 🎯 Decision Framework for Train-Test Split | إطار القرار لتقسيم البيانات

**Key Question**: What split ratio should I use?

#### Decision Tree:

```
How much data do you have?
├─ < 1000 samples → Use 70/30 or 80/20
│   └─ Why? Need enough test data for reliable evaluation
│
├─ 1000-10,000 samples → Use 80/20
│   └─ Why? Standard split, good balance
│
└─ > 10,000 samples → Use 90/10 or 95/5
    └─ Why? Large datasets, can use less for testing
```

#### Comparison Table:

| Split Ratio | When to Use | Pros | Cons | Example |
|-------------|-------------|------|------|---------|
| **70/30** | Small datasets (< 1000) | • More test data<br>• Reliable evaluation | • Less training data | Dataset with 500 samples |
| **80/20** | Medium datasets (1000-10k) | • Standard split<br>• Good balance | • May need adjustment | Dataset with 5000 samples |
| **90/10** | Large datasets (> 10k) | • More training data<br>• Still enough test data | • Less test data | Dataset with 50,000 samples |

---

### 📊 Real-World Examples | أمثلة من العالم الحقيقي

#### Example 1: House Price Prediction
- **Features**: Size (1000-5000 sq ft), Age (0-50 years), Location (A, B, C)
- **Scaling Decision**: STANDARDIZATION (different scales, normal-ish distribution)
- **Encoding Decision**: ONE-HOT for Location (nominal, 3 categories)
- **Split Decision**: 80/20 (medium dataset)
- **Reasoning**: Size and age need scaling, location needs encoding, standard split

#### Example 2: Customer Segmentation
- **Features**: Income ($30k-$200k), Education (Bachelor, Master, PhD), Age (18-65)
- **Scaling Decision**: STANDARDIZATION (different scales)
- **Encoding Decision**: LABEL ENCODING for Education (ordinal: Bachelor < Master < PhD)
- **Split Decision**: 80/20
- **Reasoning**: Income needs scaling, education has order, standard split

#### Example 3: Image Classification
- **Features**: Pixel values (0-255), Category (10 classes)
- **Scaling Decision**: NORMALIZATION (need [0,1] for neural networks)
- **Encoding Decision**: ONE-HOT for Category (nominal, 10 classes)
- **Split Decision**: 80/20
- **Reasoning**: Pixels need [0,1] range, categories need one-hot, standard split

---

### ✅ Key Takeaways | النقاط الرئيسية

1. **Standardization for normal data** - Use when data is normally distributed
2. **Normalization for bounded values** - Use when you need [0,1] range
3. **Label encoding for ordinal** - Use when categories have inherent order
4. **One-hot for nominal** - Use when categories have no order
5. **80/20 is standard** - Use for most datasets (adjust for very small/large)
6. **Always scale before encoding** - Scale numeric features first, then encode categorical
7. **Test your choices** - Try different methods and compare results

---

### 🎓 Practice Decision-Making | ممارسة اتخاذ القرار

**Scenario**: You have a dataset with:
- Income: $20,000 - $150,000 (normally distributed)
- Education: High School, Bachelor, Master, PhD (ordinal)
- City: 15 different cities (nominal)
- 5,000 samples

**Your task**: Decide preprocessing methods for each feature!

**Answers**:
1. **Income**: STANDARDIZATION (normal distribution, different scale)
2. **Education**: LABEL ENCODING (ordinal: High School < Bachelor < Master < PhD)
3. **City**: ONE-HOT ENCODING (nominal, 15 categories is acceptable)
4. **Split**: 80/20 (medium dataset, standard split)

---

**Connection to Next Steps**: 
- 📓 **Example 4: Linear Regression** - Uses preprocessed data to build models
- 📓 **Example 5: Polynomial Regression** - Extends linear regression with preprocessed data
